In [1]:
# Import packages
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.core import Model

from azureml.core import Webservice
from azureml.exceptions import WebserviceException
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

In [2]:
# Connect to workspace
ws = Workspace.from_config()

In [3]:
# Grab automl run
experiment_name = 'credit-card-default-automl'
run_id ='AutoML_f73e1768-455e-4a83-9fdd-1f05f1ea12e8'
experiment = Experiment(ws, experiment_name)
AutoML_run = AutoMLRun(experiment = experiment, run_id = run_id)

In [4]:
best_run, fitted_model = AutoML_run.get_output()

In [5]:
model_name = best_run.properties['model_name']

script_file_name = 'inference/score.py'
conda_env_file_name = 'inference/env.yml'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'inference/env.yml')

In [8]:
description = 'automl-voting-ensemble-credit-default-model'
tags = None
model = AutoML_run.register_model(model_name = model_name, description = description, tags = tags)

In [9]:
myenv = Environment.from_conda_specification(name="myenv", file_path=conda_env_file_name)
inference_config = InferenceConfig(entry_script=script_file_name, environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'type': "automl_voting_ensemble_classification"}, 
                                               description = 'service to predict credit card default')

aci_service_name = 'credit-default-service-aci'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

Running.....................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
